In [88]:
import pandas as pd
data = pd.read_csv('data.csv')
data.head()

,product_title,category,product_subcategory,brand,selling_price,original_price,product_type,product_rating,product_description,customer_reviews_count,seasonal_indicator,promotion_indicator,shipping_weight,bundle_indicator,customer_demographics,discount_offered,brand_scale
0,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,7,Spring,Yes,0.658145,Individual,Female,0.00,medium
1,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...",54,Winter,No,4.734585,Bundle,Female,0.00,medium
2,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...",31,Winter,No,2.024140,Bundle,Female,52.40,medium
3,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,49,Spring,No,3.869239,Individual,Other,15.34,medium
4,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,74,Winter,Yes,0.812489,Individual,Male,0.00,medium


In [89]:
category_stats = data.groupby(['category', 'product_subcategory']).agg({'product_rating': 'mean', 'customer_reviews_count': 'sum', 'selling_price': 'mean'}).reset_index()
category_stats = category_stats.rename(columns={'product_rating':'avg_product_rating','customer_reviews_count':'total_reviews','selling_price':'avg_selling_price'})
category_stats = category_stats.sort_values(by=['avg_product_rating', 'total_reviews'], ascending=False)
category_stats

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
34,Cleaning & Household,Detergents & Dishwash,4.269349,12864,269.750536
71,Gourmet & World Food,Oils & Vinegar,4.231590,12919,952.411297
30,Beverages,Water,4.170588,1108,153.191176
26,Beverages,Energy & Soft Drinks,4.168018,5560,150.441982
63,Gourmet & World Food,"Atta, Flours & Sooji",4.166667,550,195.500000
...,...,...,...,...,...
13,"Bakery, Cakes & Dairy",Ice Creams & Desserts,3.622917,2289,170.054167
0,Baby Care,"Atta, Flours & Sooji",3.600000,96,300.000000
79,"Kitchen, Garden & Pets",Cookware & Non Stick,3.580085,17533,1204.869774
1,Baby Care,Baby Accessories,3.145946,2169,409.648649


In [90]:
top_category = pd.pivot_table(category_stats,index='category', values=['avg_product_rating','total_reviews','avg_selling_price'],aggfunc={'avg_product_rating': 'mean',
                                      'total_reviews': 'sum',
                                      'avg_selling_price': 'mean'})

top_category= top_category.sort_values(by='avg_product_rating', ascending=False).reset_index()

In [91]:
import plotly.express as px

# Create bubble chart
fig = px.scatter(top_category, 
                 x='avg_product_rating', 
                 y='total_reviews', 
                 size='avg_selling_price',  # Size of bubbles determined by total_sales
                 color='avg_product_rating',  # Color of bubbles determined by product_rating
                 hover_data={'category': True,
                             'avg_selling_price': True,
                             'avg_product_rating': True},
                 title='Ditribution of Avg Selling Price and Avg Rating across all categories',
                 labels={'category': 'Category', 
                         'avg_selling_price': 'Avg Selling Price',
                         'avg_product_rating': 'Avg Product Rating'},
                 size_max=30,)  # Adjust maximum bubble size as needed

# Update layout
fig.update_layout(
    xaxis_title='Avg Product Rating',
    yaxis_title='Avg Selling Price'
)

# Show the plot
fig.show()


## Subcategory Analysis

In [92]:
# Find the index of the maximum average product_rating for each category
max_rating_index = category_stats.groupby('category')['avg_product_rating'].idxmax()
min_rating_index = category_stats.groupby('category')['avg_product_rating'].idxmin()

# Find the index of the maximum average customer_reviews_count for each category
max_reviews_index = category_stats.groupby('category')['total_reviews'].idxmax()
min_reviews_index = category_stats.groupby('category')['total_reviews'].idxmin()

# Extract the corresponding sub-categories with the highest average rating and customer reviews count for each category
best_subcategories_rating = category_stats.loc[max_rating_index]
low_subcategories_rating = category_stats.loc[min_rating_index]
best_subcategories_reviews = category_stats.loc[max_reviews_index]
low_subcategories_reviews = category_stats.loc[min_reviews_index]

In [93]:
best_subcategories_rating

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
3,Baby Care,Baby Food & Formula,4.165957,2363,361.152128
12,"Bakery, Cakes & Dairy",Gourmet Breads,4.105714,1881,107.455714
23,Beauty & Hygiene,Oral Care,4.124723,14320,182.256827
30,Beverages,Water,4.170588,1108,153.191176
34,Cleaning & Household,Detergents & Dishwash,4.269349,12864,269.750536
44,"Eggs, Meat & Fish",Mutton & Lamb,4.023684,841,721.110526
51,"Foodgrains, Oil & Masala",Edible Oils & Ghee,4.125605,12037,540.908710
56,Fruits & Vegetables,Cuts & Sprouts,4.000526,4875,35.368947
71,Gourmet & World Food,Oils & Vinegar,4.231590,12919,952.411297
84,"Kitchen, Garden & Pets",Pet Food & Accessories,4.138202,17233,857.348034


In [94]:
low_subcategories_rating

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
1,Baby Care,Baby Accessories,3.145946,2169,409.648649
13,"Bakery, Cakes & Dairy",Ice Creams & Desserts,3.622917,2289,170.054167
17,Beauty & Hygiene,Fragrances & Deos,3.746950,48037,893.099590
25,Beverages,Coffee,3.967978,4193,208.072360
32,Cleaning & Household,Bins & Bathroom Ware,3.760077,20064,272.686480
45,"Eggs, Meat & Fish",Pork & Other Meats,2.640000,102,263.000000
50,"Foodgrains, Oil & Masala",Dry Fruits,3.890394,10139,255.474877
60,Fruits & Vegetables,Fresh Vegetables,4.000000,8294,30.773452
66,Gourmet & World Food,Chocolates & Biscuits,3.875205,31412,269.818177
79,"Kitchen, Garden & Pets",Cookware & Non Stick,3.580085,17533,1204.869774


In [95]:
best_subcategories_reviews

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
4,Baby Care,Diapers & Wipes,4.095547,12573,741.909960
11,"Bakery, Cakes & Dairy",Dairy,4.054156,19498,149.439169
24,Beauty & Hygiene,Skin Care,3.947319,115724,412.119608
29,Beverages,Tea,4.080389,16106,269.842874
31,Cleaning & Household,All Purpose Cleaners,4.126211,24219,271.590611
46,"Eggs, Meat & Fish","Sausages, Bacon & Salami",4.023270,7610,282.237736
52,"Foodgrains, Oil & Masala",Masalas & Spices,4.094094,46249,90.488481
60,Fruits & Vegetables,Fresh Vegetables,4.000000,8294,30.773452
75,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",3.988333,42189,227.910952
86,"Kitchen, Garden & Pets",Storage & Accessories,3.827833,49877,303.489655


In [96]:
low_subcategories_reviews

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
0,Baby Care,"Atta, Flours & Sooji",3.600000,96,300.000000
7,"Bakery, Cakes & Dairy",Bakery Snacks,3.938000,1315,104.448400
22,Beauty & Hygiene,Mothers & Maternity,3.885714,332,333.714286
30,Beverages,Water,4.170588,1108,153.191176
33,Cleaning & Household,Car & Shoe Care,3.888542,2056,198.100417
43,"Eggs, Meat & Fish",Marinades,4.000000,48,225.000000
48,"Foodgrains, Oil & Masala",Baby Food & Formula,4.000000,52,225.000000
58,Fruits & Vegetables,"Flower Bouquets, Bunches",4.000000,473,83.250000
70,Gourmet & World Food,Mutton & Lamb,4.050000,97,619.000000
78,"Kitchen, Garden & Pets",Bakeware,3.650000,2288,214.666667


In [97]:
merged_df = pd.concat([best_subcategories_rating, best_subcategories_reviews], ignore_index=True)
merged_df_low = pd.concat([low_subcategories_rating, low_subcategories_reviews], ignore_index=True)

# Sort the merged dataframe by category and then by either product_rating or customer_reviews_count
merged_df.sort_values(by=['category', 'avg_product_rating', 'total_reviews'], ascending=[True, False, False], inplace=True)
merged_df_low.sort_values(by=['category', 'avg_product_rating', 'total_reviews'], ascending=[True, False, False], inplace=True)

merged_df

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
0,Baby Care,Baby Food & Formula,4.165957,2363,361.152128
11,Baby Care,Diapers & Wipes,4.095547,12573,741.909960
1,"Bakery, Cakes & Dairy",Gourmet Breads,4.105714,1881,107.455714
12,"Bakery, Cakes & Dairy",Dairy,4.054156,19498,149.439169
2,Beauty & Hygiene,Oral Care,4.124723,14320,182.256827
13,Beauty & Hygiene,Skin Care,3.947319,115724,412.119608
3,Beverages,Water,4.170588,1108,153.191176
14,Beverages,Tea,4.080389,16106,269.842874
4,Cleaning & Household,Detergents & Dishwash,4.269349,12864,269.750536
15,Cleaning & Household,All Purpose Cleaners,4.126211,24219,271.590611


In [98]:
merged_df_low

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
11,Baby Care,"Atta, Flours & Sooji",3.600000,96,300.000000
0,Baby Care,Baby Accessories,3.145946,2169,409.648649
12,"Bakery, Cakes & Dairy",Bakery Snacks,3.938000,1315,104.448400
1,"Bakery, Cakes & Dairy",Ice Creams & Desserts,3.622917,2289,170.054167
13,Beauty & Hygiene,Mothers & Maternity,3.885714,332,333.714286
2,Beauty & Hygiene,Fragrances & Deos,3.746950,48037,893.099590
14,Beverages,Water,4.170588,1108,153.191176
3,Beverages,Coffee,3.967978,4193,208.072360
15,Cleaning & Household,Car & Shoe Care,3.888542,2056,198.100417
4,Cleaning & Household,Bins & Bathroom Ware,3.760077,20064,272.686480


### Combination of category and sub-category list to be shown in ppt

In [99]:
import plotly.graph_objs as go

# Create figure for best product rating
fig1 = go.Figure(go.Bar(
    y=best_subcategories_rating['category'] + ' - ' + best_subcategories_rating['product_subcategory'],
    x=best_subcategories_rating['avg_product_rating'],
    name='Average Product Rating',
    orientation='h',
    marker=dict(color='rgba(255, 153, 51, 0.7)'),
    text=best_subcategories_rating['avg_product_rating'].round(2),
    textposition='auto'
))

fig1.update_layout(
    title='Subcategories with Best Average Rating for Each Category',
    yaxis=dict(title='Category - Subcategory'),
    xaxis=dict(title='Average Rating', range=[0, best_subcategories_rating['avg_product_rating'].max() + 1]), 
    bargap=0.2,
    margin=dict(l=150),
)

# Create figure for best customer reviews count
fig2 = go.Figure(go.Bar(
    y=best_subcategories_reviews['category'] + ' - ' + best_subcategories_reviews['product_subcategory'],
    x=best_subcategories_reviews['total_reviews'],
    name='Total Customer Reviews Count',
    orientation='h',
    marker=dict(color='rgba(51, 153, 255, 0.7)'),
    text=best_subcategories_reviews['total_reviews'],
    textposition='auto'
))

fig2.update_layout(
    title='Subcategories with High Reviews for Each Category',
    yaxis=dict(title='Category - Subcategory'),
    xaxis=dict(title='Total Reviews Count'),
    bargap=0.2,
    margin=dict(l=150)
)

# Show the plots one below another
fig1.show()
fig2.show()


In [100]:
low_subcategories_rating

,category,product_subcategory,avg_product_rating,total_reviews,avg_selling_price
1,Baby Care,Baby Accessories,3.145946,2169,409.648649
13,"Bakery, Cakes & Dairy",Ice Creams & Desserts,3.622917,2289,170.054167
17,Beauty & Hygiene,Fragrances & Deos,3.746950,48037,893.099590
25,Beverages,Coffee,3.967978,4193,208.072360
32,Cleaning & Household,Bins & Bathroom Ware,3.760077,20064,272.686480
45,"Eggs, Meat & Fish",Pork & Other Meats,2.640000,102,263.000000
50,"Foodgrains, Oil & Masala",Dry Fruits,3.890394,10139,255.474877
60,Fruits & Vegetables,Fresh Vegetables,4.000000,8294,30.773452
66,Gourmet & World Food,Chocolates & Biscuits,3.875205,31412,269.818177
79,"Kitchen, Garden & Pets",Cookware & Non Stick,3.580085,17533,1204.869774


In [101]:
import plotly.graph_objs as go

# Create figure for best product rating
fig1 = go.Figure(go.Bar(
    y=low_subcategories_rating['category'] + ' - ' + low_subcategories_rating['product_subcategory'],
    x=low_subcategories_rating['avg_product_rating'],
    name='Average Product Rating',
    orientation='h',
    marker=dict(color='rgba(255, 153, 51, 0.7)'),
    text=low_subcategories_rating['avg_product_rating'].round(2),
    textposition='auto'
))

fig1.update_layout(
    title='Subcategories with Low Average Rating for Each Category',
    yaxis=dict(title='Category - Subcategory'),
    xaxis=dict(title='Average Rating', range=[0, low_subcategories_rating['avg_product_rating'].max() + 1]), 
    bargap=0.2,
    margin=dict(l=150),
)

# Create figure for best customer reviews count
fig2 = go.Figure(go.Bar(
    y=low_subcategories_reviews['category'] + ' - ' + low_subcategories_reviews['product_subcategory'],
    x=low_subcategories_reviews['total_reviews'],
    name='Total Customer Reviews Count',
    orientation='h',
    marker=dict(color='rgba(51, 153, 255, 0.7)'),
    text=low_subcategories_reviews['total_reviews'],
    textposition='auto'
))

fig2.update_layout(
    title='Subcategories with Low Reviews for Each Category',
    yaxis=dict(title='Category - Subcategory'),
    xaxis=dict(title='Total Reviews Count'),
    bargap=0.2,
    margin=dict(l=150)
)

# Show the plots one below another
fig1.show()
fig2.show()
